# CLI automation with Cisco pyATS 🚀

**pyATS** (Python Automated Test System) is a Cisco-developed Python framework for **network test automation and verification**. It helps network engineers and developers **automate configuration, testing, and validation** on real and virtual devices.

---

### 🧩 Components

- **pyATS Core** 🔹  
  The foundation for network automation, managing devices, connections, and test execution.

- **Genie** 🧪  
  Cisco’s parsing and abstraction library. Converts raw device output into structured data (facts) and provides configuration models.

- **Unicon** 🔌  
  Handles **device connectivity** (SSH, Telnet, console) and session management.

- **Testbeds** 🗺️  
  YAML-based definitions of devices, connections, credentials, and topology for consistent automation.

- **Ops and Conf Libraries** ⚙️  
  - **Ops**: Learn device state, interfaces, routing tables, etc.  
  - **Conf**: Build or rollback configurations programmatically.

---

### 🌟 Benefits

- ✅ Automate repetitive network tasks  
- ✅ Test configurations and verify device state  
- ✅ Generate dry-run config previews and diffs  
- ✅ Multi-vendor and multi-platform support (with Genie)  
- ✅ Safe rollback and candidate configuration support on IOS-XR and other platforms  

---

🔁 First of all, create a virtual environment with the following commands:

In [ ]:
!python3 -m venv .venv && source .venv/bin/activate
!pip install -r requirements.txt

### 1️⃣ Create my testbed
Now, let's create a `topology.yaml` file where we specify which are all our devices, and how to connect to them.</br>

### 2️⃣ Connect to my device and retrieve parsed configs
Now, let's roll our sleeves a little bit and get hands-on coding!</br>
Let's connect to our device using the info in our `testbed.yaml` file.</br>
pyATS will use the `unicon` library and the information from the yaml file to attempt to connect to the device `sandbox1`.

In [1]:
import json
from pyats.topology import loader

In [2]:
testbed = loader.load('pyATS/testbed.yaml')
device = testbed.devices['sandbox1']

device.connect()
print("✅ Connected successfully to", device.name)


2025-11-13 12:56:46,881: %UNICON-INFO: +++ sandbox1 logfile sandbox1-cli-1763035006.log +++

2025-11-13 12:56:46,882: %UNICON-INFO: +++ Unicon plugin iosxr (unicon.plugins.iosxr) +++

Hello there! Hoping you are having a great day
... Welcome to 'ios',
your favorite CISCO.IOSXR.IOSXR Sandbox



2025-11-13 12:56:48,302: %UNICON-INFO: +++ connection to spawn: ssh -l admin 131.226.217.150 -p 22, id: 4646036688 +++

2025-11-13 12:56:48,304: %UNICON-INFO: connection to sandbox1
(admin@131.226.217.150) Password: 


RP/0/RP0/CPU0:ios#

2025-11-13 12:56:49,220: %UNICON-INFO: Storing credentials from default as current_credentials

2025-11-13 12:56:49,223: %UNICON-WARNING: Invalid hostname detected: found ios, expected sandbox1

2025-11-13 12:56:49,225: %UNICON-INFO: +++ initializing handle +++

2025-11-13 12:56:49,426: %UNICON-INFO: +++ sandbox1 with via 'cli': executing command 'terminal length 0' +++
terminal length 0
Thu Nov 13 12:44:13.685 UTC
RP/0/RP0/CPU0:ios#

2025-11-13 12:56:49,952: 

🔌 Lovely. Now, we want to get **information about the interfaces in this device**.</br>
**Let's use [the built-in Genie parsers! 🧞](https://pubhub.devnetcloud.com/media/genie-feature-browser/docs/#/parsers)** to get this output in a nice JSON.

In [4]:
genie_parsed_interfaces = device.parse("show ip interface brief")
print(f"\n\n\n\n\n{json.dumps(genie_parsed_interfaces,indent=4)}")


2025-11-13 13:02:19,189: %UNICON-INFO: +++ sandbox1 with via 'cli': executing command 'show ip interface brief' +++
show ip interface brief
Thu Nov 13 12:49:43.447 UTC

Interface                      IP-Address      Status          Protocol Vrf-Name
Loopback0                      unassigned      Up              Up       default 
Loopback1                      unassigned      Up              Up       default 
Loopback2                      unassigned      Up              Up       default 
Loopback3                      unassigned      Up              Up       default 
Loopback4                      unassigned      Up              Up       default 
Loopback5                      unassigned      Up              Up       default 
Loopback6                      unassigned      Up              Up       default 
Loopback7                      unassigned      Up              Up       default 
Loopback8                      unassigned      Up              Up       default 
Loopback9           

### 3️⃣ Create and rollback a configuration
🤖 Let's use [pyATS built-in config classes](https://pubhub.devnetcloud.com/media/genie-feature-browser/docs/#/models) to create a new Lopback interface! pyATS will figure out the specifics for our device. We just provide the essential information!

In [5]:
from genie.conf.base import Interface

new_iosxr_interface = Interface(name="Loopback503", device=device)
new_iosxr_interface.description = "TechTilesDemo02"
rendered_config = new_iosxr_interface.build_config()
print(rendered_config)


2025-11-13 13:03:28,020: %UNICON-INFO: +++ sandbox1 with via 'cli': configure +++
configure terminal
Thu Nov 13 12:50:52.471 UTC
RP/0/RP0/CPU0:ios(config)#interface Loopback503
RP/0/RP0/CPU0:ios(config-if)# description TechTilesDemo02
RP/0/RP0/CPU0:ios(config-if)# exit
RP/0/RP0/CPU0:ios(config)#commit
Thu Nov 13 12:50:53.461 UTC
RP/0/RP0/CPU0:ios(config)#end
RP/0/RP0/CPU0:ios#
None


🔥 But now, we want to wipe clean our changes. We can easily retrieve the configurations applied by pyATS and remove them!

In [6]:
iface = device.interfaces.get("Loopback503")
rollback_config = iface.build_unconfig()
print(rollback_config)


2025-11-13 13:03:53,871: %UNICON-INFO: +++ sandbox1 with via 'cli': configure +++
configure terminal
Thu Nov 13 12:51:18.330 UTC
RP/0/RP0/CPU0:ios(config)#no interface Loopback503
RP/0/RP0/CPU0:ios(config)#commit
Thu Nov 13 12:51:18.938 UTC
RP/0/RP0/CPU0:ios(config)#end
RP/0/RP0/CPU0:ios#
None
